In [ ]:
!pip install sortedcontainers
!pip install git+https://github.com/ihler/pyGMs.git
!pip install pyro-ppl



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/ihler/pyGMs.git to /tmp/pip-req-build-g1dl1y2m
  Running command git clone --filter=blob:none --quiet https://github.com/ihler/pyGMs.git /tmp/pip-req-build-g1dl1y2m
  Resolved https://github.com/ihler/pyGMs.git to commit 600d48493b868df42da2d6119ee14020c665c59d
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import matplotlib.pyplot as plt

import pyGMs as gm
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist
import pyro.poutine as poutine

import pandas as pd

import requests                                      # reading data
from io import StringIO
import re                                            # regular expressions

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 123
pyro.set_rng_seed(seed)
#np.random.seed(seed);
#torch.manual_seed(seed);

In [ ]:
#getting info
df = pd.read_csv('games.csv')


#print(df['white_id'])
white_ids = df['white_id']
black_ids =  df["black_id"]
winners = df["winner"]

print(white_ids[:5])

0         bourgris
1             a-00
2           ischia
3    daniamurashov
4        nik221107
Name: white_id, dtype: object


In [ ]:
uniqueppl = white_ids.tolist()
uniqueppl.extend(black_ids.tolist())
#print(len(set(uniqueppl)))




uppl = np.array(list(set(uniqueppl)))#set of unique people

gamesplayed = []
for game in range(0,len(df)):
    p1 = np.argwhere(uppl == df.iloc[game].white_id)[0][0]
    p2 =np.argwhere(uppl == df.iloc[game].black_id)[0][0]

    winner = 0
    if df.iloc[game].winner == "white":
        winner = 1
    if df.iloc[game].winner != "draw":
        gamesplayed.append([p1,p2,winner])

In [ ]:
def model(matches):
    nPlayers = max([max(m[0],m[1]) for m in matches])+1
    X = [pyro.sample(f'X{i}', dist.Normal(torch.tensor([0.0]), torch.tensor([1.0]))) for i in range(nPlayers)]
    wins = [pyro.sample(f'w{i}', dist.Bernoulli(logits=X[m[0]]-X[m[1]]), obs=torch.tensor(m[2]*
    1.)) for i,m in enumerate(matches)]


In [ ]:
matches = gamesplayed[:5]

nuts_kernel = pyro.infer.NUTS(model, jit_compile=False)
mcmc = pyro.infer.MCMC(nuts_kernel,
    num_samples=2000,
    # warmup_steps=100,
    warmup_steps=50,
    num_chains=1)
mcmc.run(matches)

Warmup:   0%|          | 5/2050 [02:58, 38.45s/it, step size=5.78e-02, acc. prob=0.408]

KeyboardInterrupt: 

In [ ]:
hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}
xsc = np.hstack( tuple(hmc_samples['X{}'.format(i)] for i in range(4)) )
print("Pr[0>1]: ",(xsc[:,0] > xsc[:,1]).mean())
print("Pr[0>3]: ",(xsc[:,0] > xsc[:,3]).mean())

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f17de2b6-daef-4780-abae-640fa6a3dc63' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>